In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import json

In [2]:
with open("datasets/user-wallet-transactions.json") as f:
    data = json.load(f)
data[:2]

[{'_id': {'$oid': '681d38fed63812d4655f571a'},
  'userWallet': '0x00000000001accfa9cef68cf5371a23025b6d4b6',
  'network': 'polygon',
  'protocol': 'aave_v2',
  'txHash': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190',
  'logId': '0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6d2e66e6b0e3b2e8d4190_Deposit',
  'timestamp': 1629178166,
  'blockNumber': 1629178166,
  'action': 'deposit',
  'actionData': {'type': 'Deposit',
   'amount': '2000000000',
   'assetSymbol': 'USDC',
   'assetPriceUSD': '0.9938318274296357543568636362026045',
   'poolId': '0x2791bca1f2de4661ed88a30c99a7a9449aa84174',
   'userId': '0x00000000001accfa9cef68cf5371a23025b6d4b6'},
  '__v': 0,
  'createdAt': {'$date': '2025-05-08T23:06:39.465Z'},
  'updatedAt': {'$date': '2025-05-08T23:06:39.465Z'}},
 {'_id': {'$oid': '681aa70dd6df53021cc6f3c0'},
  'userWallet': '0x000000000051d07a4fb3bd10121a343d85818da6',
  'network': 'polygon',
  'protocol': 'aave_v2',
  'txHash': '0xe6fc162c86b2928b0ba9b82bda6727

In [3]:
def parse_tx(tx):
    return {
        "wallet": tx.get("userWallet"),
        "network": tx.get("network"),
        "protocol": tx.get("protocol"),
        "action": tx.get("action"),
        "amount_raw": tx.get("actionData", {}).get("amount"),
        "asset": tx.get("actionData", {}).get("assetSymbol"),
        "asset_price_usd": tx.get("actionData", {}).get("assetPriceUSD"),
        "user_id": tx.get("actionData", {}).get("userId"),
        "to_id": tx.get("actionData", {}).get("toId"),
    }

# Apply to all tx records
parsed = [parse_tx(tx) for tx in data]

df = pd.DataFrame(parsed)


#### Converting all the amounts of different asset types to usd 

In [4]:
# Convert asset_price_usd and asset_price_usd to float
df["amount_raw"] = pd.to_numeric(df["amount_raw"], errors='coerce')

df["asset_price_usd"] = pd.to_numeric(df["asset_price_usd"], errors='coerce')

print(df.dtypes)

wallet              object
network             object
protocol            object
action              object
amount_raw         float64
asset               object
asset_price_usd    float64
user_id             object
to_id               object
dtype: object


In [5]:
asset_decimals = {
    "USDC": 6,
    "WMATIC": 18,
    "DAI": 18,
    "WBTC": 8,
    "WETH": 18,
    "USDT": 6,
    "WPOL": 18,     
    "AAVE": 18          
}

# function to do so

def convert_raw_to_token(row):
    if row["asset"] == "":
        return 0.0  # No amount to calculate for liquidation
    decimals = asset_decimals.get(row["asset"], 18)
    try:
        return int(row["amount_raw"]) / (10 ** decimals)
    except:
        return 0.0
        
df["no_token"] = df.apply(convert_raw_to_token, axis=1)

In [6]:
df.head()

,wallet,network,protocol,action,amount_raw,asset,asset_price_usd,user_id,to_id,no_token
0,0x00000000001accfa9cef68cf5371a23025b6d4b6,polygon,aave_v2,deposit,2.000000e+09,USDC,0.993832,0x00000000001accfa9cef68cf5371a23025b6d4b6,None,2000.000000
1,0x000000000051d07a4fb3bd10121a343d85818da6,polygon,aave_v2,deposit,1.450000e+20,WMATIC,1.970307,0x000000000051d07a4fb3bd10121a343d85818da6,None,145.000000
2,0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,deposit,1.000000e+15,WMATIC,0.922377,0x000000000096026fb41fc39f9875d164bd82e2dc,None,0.001000
3,0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,deposit,4.000000e+15,WMATIC,1.057483,0x000000000096026fb41fc39f9875d164bd82e2dc,None,0.004000
4,0x0000000000e189dd664b9ab08a33c4839953852c,polygon,aave_v2,redeemunderlying,5.015480e+05,USDC,1.002130,0x7f90122bf0700f9e7e1f688fe926940e8839f353,0x0000000000e189dd664b9ab08a33c4839953852c,0.501548


In [7]:
df.to_csv("datasets//parsed_data.csv", index=False)